<a href="https://colab.research.google.com/github/vkdnjdldnjsw/2019_cau_oss_hackathon/blob/master/assignment/10/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvcc --version
!pip install cupy-cuda101

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [0]:
import numpy as np
file_data   = "./drive/My Drive/10/mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)
count       = 0     # count for the number of images

def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

list_image  = np.empty((num_image, size_row * size_col), dtype=float)
list_label  = np.empty(num_image, dtype=int)

for line in data:

    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector)

    list_label[count]    = label
    list_image[count]    = im_vector

    count += 1

train_image = list_image[:1000]
train_label = list_label[:1000]

test_image = list_image[1000:]
test_label = list_label[1000:]



In [0]:
def get_init_thetas(channel_nums):
  thetas = []
  for i in range(len(channel_nums)-1):
    devi = (2 / (channel_nums[i+1] + channel_nums[i] + 1)) ** 0.5
    temp_theta = np.random.normal(0, devi, (channel_nums[i] + 1) * channel_nums[i+1])
    temp_theta = temp_theta.reshape([-1, channel_nums[i+1], channel_nums[i] + 1])
    thetas.append(temp_theta)
  return thetas

def sigmoid(z):
  return 1 / (1 + np.exp(-z))

def propagation(thetas, xs):
  temp_xs = xs
  results = []
  for theta in thetas:
    temp_xs = np.concatenate((temp_xs, np.zeros([len(temp_xs), 1])), axis=1)
    results.append(temp_xs)
    temp_xs = temp_xs.reshape([temp_xs.shape[0], 1, temp_xs.shape[1]])
    num_batch = 100
    tempA = []
    tempA.append(np.sum(temp_xs[:int(len(temp_xs)/num_batch)] * theta, axis=2))
    for i in range(1, num_batch-1):
      tempA.append(np.sum(temp_xs[int(len(temp_xs)* i/num_batch):int(len(temp_xs)* (i+1)/num_batch)] * theta, axis=2))
    tempA.append(np.sum(temp_xs[int(len(temp_xs)* (i+1)/num_batch):] * theta, axis=2))
    temp = np.concatenate(tempA, axis=0)
    temp_xs = sigmoid(temp)
    # temp_xs = sigmoid(np.sum(temp_xs * theta, axis=2))
  results.append(temp_xs)
  return (temp_xs, results)

def objectiveFunction(labels, results, thetas, controlParameter):
  label_one_hot = np.eye(channel_nums[-1])[labels]
  temp = -1 * label_one_hot * np.log(results[-1]) - (1 - label_one_hot) * np.log(1 - results[-1])
  temp = np.sum(temp, axis=1)
  temp = np.mean(temp)
  temp2 = 0
  n = 0
  for theta in thetas:
    temp2 += np.sum(theta**2)
    n += len(theta.flatten())
  temp += controlParameter * 0.5 * temp2 / n
  return temp



In [0]:
def backpropa(labels, thetas, results, lr, controlParameter):
  label_one_hot = np.eye(channel_nums[-1])[labels]

  dp = []
  new_thetas = []

  numOfThetas = 0
  for theta in thetas:
    numOfThetas += len(theta.flatten())
  
  #for last theta
  temp =  results[-1] - label_one_hot
  dp.append(temp)
  temp = temp.reshape([temp.shape[0], temp.shape[1], 1]) * results[-2].reshape([results[-2].shape[0], 1, results[-2].shape[1]])
  temp = np.mean(temp, axis=0)
  temp += controlParameter * np.sum(thetas[-1]) / numOfThetas
  new_thetas.append(thetas[-1] - lr * temp)

  for i in range(1, len(thetas)):
    temp = results[-1 * (i + 1)][:,:-1]
    temp = temp * (1 - temp)
    dp_shape = []
    dp_shape.append(temp.shape[0])
    dp_shape.append(1)
    for pre_dp_shape in range(1, len(temp.shape)):
      dp_shape.append(temp.shape[pre_dp_shape])
    temp = thetas[-1 * i][:,:,:-1] * temp.reshape(dp_shape)
    dp.append(temp)
    for j in range(i):
      dp_shape = []
      dp_shape.append(temp.shape[0])
      if(i-j != 1):
        dp_shape.append(1)
      for pre_dp_shape in range(1, len(temp.shape)):
        dp_shape.append(temp.shape[pre_dp_shape])
      temp = temp.reshape(dp_shape)
      dp_shape = []
      for pre_dp_shape in dp[i-1-j].shape:
        dp_shape.append(pre_dp_shape)
      for add_one_num in range(len(temp.shape) - len(dp_shape)):
        dp_shape.append(1)
      temp = dp[i-1-j].reshape(dp_shape) * temp
      temp = np.sum(temp, axis=len(temp.shape)-2)
    temp = temp.reshape([temp.shape[0], temp.shape[1], 1])
    temp2 = results[-1 * (i + 2)].reshape(results[-1 * (i + 2)].shape[0], 1, results[-1 * (i + 2)].shape[1])
    temp = temp.reshape(temp.shape[0], temp.shape[1], 1)
    half_num = len(temp) / 2
    temp3 = np.mean(temp[:half_num] * temp2[:half_num], axis=0)
    temp3 += np.mean(temp[half_num:] * temp2[half_num:], axis=0)
    temp = temp3 / 2
    temp += controlParameter * np.sum(thetas[-1 * (i + 1)]) / numOfThetas
    new_thetas.append(thetas[-1 * (i + 1)] - lr * temp)
  new_thetas.reverse()
  return new_thetas
 

In [0]:
#for big
def backpropa(labels, thetas, results, lr, controlParameter):
  label_one_hot = np.eye(channel_nums[-1])[labels]

  dp = []
  new_thetas = []
  num_batch = 100

  numOfThetas = 0
  for theta in thetas:
    numOfThetas += len(theta.flatten())
  
  #for last theta
  temp =  results[-1] - label_one_hot
  dp.append(temp)
  temp = temp.reshape([temp.shape[0], temp.shape[1], 1]) * results[-2].reshape([results[-2].shape[0], 1, results[-2].shape[1]])
  temp = np.mean(temp, axis=0)
  temp += controlParameter * np.sum(thetas[-1]) / numOfThetas
  new_thetas.append(thetas[-1] - lr * temp)

  for i in range(1, len(thetas)):
    temp = results[-1 * (i + 1)][:,:-1]
    temp = temp * (1 - temp)
    dp_shape = []
    dp_shape.append(temp.shape[0])
    dp_shape.append(1)
    for pre_dp_shape in range(1, len(temp.shape)):
      dp_shape.append(temp.shape[pre_dp_shape])
    temp = thetas[-1 * i][:,:,:-1] * temp.reshape(dp_shape)
    dp.append(temp)
    for j in range(i):
      dp_shape = []
      dp_shape.append(temp.shape[0])
      if(i-j != 1):
        dp_shape.append(1)
      for pre_dp_shape in range(1, len(temp.shape)):
        dp_shape.append(temp.shape[pre_dp_shape])
      temp = temp.reshape(dp_shape)
      dp_shape = []
      for pre_dp_shape in dp[i-1-j].shape:
        dp_shape.append(pre_dp_shape)
      for add_one_num in range(len(temp.shape) - len(dp_shape)):
        dp_shape.append(1)
      tempA = []
      temp2 = dp[i-1-j].reshape(dp_shape)
      tempA.append(np.sum(temp2[:int(len(temp2)/num_batch)] * temp[:int(len(temp)/num_batch)], axis=len(temp.shape)-2))
      for index in range(1, num_batch-1):
        tempA.append(np.sum(temp2[int(len(temp2) * index/num_batch):int(len(temp2) * (index + 1) /num_batch)] * temp[int(len(temp) * index/num_batch):int(len(temp) * (index + 1)/num_batch)], axis=len(temp.shape)-2))
      tempA.append(np.sum(temp2[int(len(temp2) * (index+1)/num_batch):] * temp[int(len(temp) * (index+1)/num_batch):], axis=len(temp.shape)-2))
      temp = np.concatenate(tempA, axis=0)
    temp = temp.reshape([temp.shape[0], temp.shape[1], 1])
    temp2 = results[-1 * (i + 2)].reshape(results[-1 * (i + 2)].shape[0], 1, results[-1 * (i + 2)].shape[1])
    temp = temp.reshape(temp.shape[0], temp.shape[1], 1)
    half_num = len(temp) / 2
    temp3 = np.mean(temp[:half_num] * temp2[:half_num], axis=0)
    temp3 += np.mean(temp[half_num:] * temp2[half_num:], axis=0)
    temp = temp3 / 2
    temp += controlParameter * np.sum(thetas[-1 * (i + 1)]) / numOfThetas
    new_thetas.append(thetas[-1 * (i + 1)] - lr * temp)
  new_thetas.reverse()
  return new_thetas
 

In [0]:
import time
def process(thetas, train_image, train_label, test_image, test_label, channel_nums, lr, errors, accureacies, pbar, itor, controlParameter):
  if np.__name__ == 'cupy':
    train_image = np.array(train_image)
    train_label = np.array(train_label)
    test_image = np.array(test_image)
    test_label = np.array(test_label)
    for index in range(len(thetas)):
      thetas[index] = np.array(thetas[index])
  prev_error = -2
  train_error = -1
  count = 0
  while(train_error != prev_error and count < itor):
  # for i in range(itor):
    start = time.time()
    prev_error = train_error
    (train_predict, train_results) = propagation(thetas, train_image)
    train_predict = np.argmax(train_predict, axis=1)
    train_accurcay = np.mean(train_predict == train_label)
    (test_predict, test_results) = propagation(thetas, test_image)
    test_predict = np.argmax(test_predict, axis=1)
    test_accurcay = np.mean(test_predict == test_label)
    train_error = objectiveFunction(train_label, train_results, thetas, controlParameter)
    test_error = objectiveFunction(test_label, test_results, thetas, controlParameter)

    errors[0].append(train_error)
    errors[1].append(test_error)
    accureacies[0].append(train_accurcay * 100)
    accureacies[1].append(test_accurcay * 100)

    thetas = backpropa(train_label, thetas, train_results, lr, controlParameter)
    count += 1
    pbar.update(1)
    print(len(errors[0]), str(np.around(time.time() - start, 2)) + 's' \
          ,'/' , str(np.around(train_accurcay * 100, 2)) + "%", train_error \
          ,'/' , str(np.around(test_accurcay * 100, 2)) + "%", test_error)

  if np.__name__ == 'cupy':
    for i in range(len(thetas)):
      thetas[i] = np.asnumpy(thetas[i])
    test_predict = np.asnumpy(test_predict)
  return [test_predict, thetas]

In [0]:
import pickle
length = str(5000)
with open('./drive/My Drive/10/accureacies/'+length+'.pk', 'rb') as f:
  accureacies = pickle.load(f)
with open('./drive/My Drive/10/errors/'+length+'.pk', 'rb') as f:
  errors = pickle.load(f)
with open('./drive/My Drive/10/thetas/'+length+'.pk', 'rb') as f:
  thetas = pickle.load(f)
with open('./drive/My Drive/10/test_predict/'+length+'.pk', 'rb') as f:
  test_predict = pickle.load(f)
channel_nums = [size_row * size_col]
for theta in thetas:
  channel_nums.append(theta.shape[1])

In [0]:
#5002
print(channel_nums)
print(accureacies[1][-1])

87.78


In [0]:
#5001
print(channel_nums)
print(accureacies[1][-1])

[784, 128, 256, 10]
88.63


In [6]:
#5000
print(channel_nums)
print(accureacies[1][-1])

[784, 128, 250, 10]
88.93333333333334


In [0]:
channel_nums = [size_row * size_col, 128, 250, 10]
thetas = get_init_thetas(channel_nums)
errors = [[], []]
accureacies = [[], []]

In [0]:
import pickle
from tqdm.notebook import tqdm
import cupy as np
# import numpy as np
itor = 5000
itor_itor = 1
pbar = tqdm(total=itor * itor_itor)
for i in range(itor_itor):
  [test_predict, thetas] = process(thetas, train_image, train_label, test_image, test_label, channel_nums, 1, errors, accureacies, pbar, itor, 0.25)
  # print("save")
  # length = str(len(errors[0]))
  # with open('./drive/My Drive/10/accureacies/'+length+'.pk', 'wb') as f:
  #   pickle.dump(accureacies, f)
  # with open('./drive/My Drive/10/errors/'+length+'.pk', 'wb') as f:
  #   pickle.dump(errors, f)
  # with open('./drive/My Drive/10/thetas/'+length+'.pk', 'wb') as f:
  #   pickle.dump(thetas, f)
  # with open('./drive/My Drive/10/test_predict/'+length+'.pk', 'wb') as f:
  #   pickle.dump(test_predict, f)
  # print("save done", length)
pbar.close()
print(len(errors[0]), len(errors[1]), len(accureacies[0]), len(accureacies[1]))

In [0]:
channel_nums = [size_row * size_col, 130, 250, 10]
thetas = get_init_thetas(channel_nums)
errors = [[], []]
accureacies = [[], []]

In [0]:
import pickle
from tqdm.notebook import tqdm
import cupy as np
# import numpy as np
itor = 5000
itor_itor = 1
pbar = tqdm(total=itor * itor_itor)
for i in range(itor_itor):
  [test_predict, thetas] = process(thetas, train_image, train_label, test_image, test_label, channel_nums, 1, errors, accureacies, pbar, itor, 0.5)
  # print("save")
  # length = str(len(errors[0]))
  # with open('./drive/My Drive/10/accureacies/'+length+'.pk', 'wb') as f:
  #   pickle.dump(accureacies, f)
  # with open('./drive/My Drive/10/errors/'+length+'.pk', 'wb') as f:
  #   pickle.dump(errors, f)
  # with open('./drive/My Drive/10/thetas/'+length+'.pk', 'wb') as f:
  #   pickle.dump(thetas, f)
  # with open('./drive/My Drive/10/test_predict/'+length+'.pk', 'wb') as f:
  #   pickle.dump(test_predict, f)
  # print("save done", length)
pbar.close()
print(len(errors[0]), len(errors[1]), len(accureacies[0]), len(accureacies[1]))

5001 0.59s / 100.0% 0.0059410819954273004 / 88.93% 1.0729235990229085
5002 0.43s / 100.0% 0.00594063280746894 / 88.93% 1.0729492342702303
5003 0.43s / 100.0% 0.005940183880915395 / 88.93% 1.0729748636244572
5004 0.43s / 100.0% 0.005939735215564073 / 88.93% 1.0730004870880216
5005 0.43s / 100.0% 0.0059392868112126095 / 88.93% 1.0730261046641796
5006 0.43s / 100.0% 0.005938838667658847 / 88.93% 1.0730517163549578
5007 0.43s / 100.0% 0.005938390784700829 / 88.93% 1.0730773221629888
5008 0.43s / 100.0% 0.0059379431621367695 / 88.93% 1.0731029220910169
5009 0.43s / 100.0% 0.0059374957997651405 / 88.93% 1.0731285161418178
5010 0.43s / 100.0% 0.0059370486973845515 / 88.93% 1.0731541043178265
5011 0.43s / 100.0% 0.005936601854793842 / 88.93% 1.073179686621322
5012 0.43s / 100.0% 0.0059361552717920344 / 88.93% 1.0732052630552418
5013 0.43s / 100.0% 0.005935708948178386 / 88.93% 1.0732308336221144
5014 0.43s / 100.0% 0.00593526288375231 / 88.93% 1.0732563983247099
5015 0.43s / 100.0% 0.005934817

In [0]:
print("save")
length = str(len(errors[0]))
with open('./drive/My Drive/10/accureacies/'+length+'.pk', 'wb') as f:
  pickle.dump(accureacies, f)
with open('./drive/My Drive/10/errors/'+length+'.pk', 'wb') as f:
  pickle.dump(errors, f)
with open('./drive/My Drive/10/thetas/'+length+'.pk', 'wb') as f:
  pickle.dump(thetas, f)
with open('./drive/My Drive/10/test_predict/'+length+'.pk', 'wb') as f:
  pickle.dump(test_predict, f)
print("save done", length)

save
save done 5000


In [0]:
import numpy as np
correct_index = np.where(test_predict == test_label)[0]
wrong_index = np.where(test_predict != test_label)[0]

# Plot

In [0]:
import matplotlib.pyplot as plt

time_points = range(len(errors[0]))
plt.figure(figsize=(8, 8))
plt.title("loss curve")
plt.xlabel("times")
plt.ylabel("loss")
plt.plot(time_points, errors[0], 'b.--')
plt.plot(time_points, errors[1], 'r.--')
plt.show()

In [0]:
import matplotlib.pyplot as plt

time_points = range(len(accureacies[0]))
plt.figure(figsize=(8, 8))
plt.title("accuracy curve")
plt.xlabel("times")
plt.ylabel("accuracy")
plt.plot(time_points, accureacies[0], 'b.--')
plt.plot(time_points, accureacies[1], 'r.--')
plt.show()

In [0]:
print("final training accuracy :", str(accureacies[0][-1]) + "%")
print("final testing accuracy :", str(accureacies[1][-1]) + "%")

In [0]:
import matplotlib.pyplot as plt

print("correct")
for i in range(10):
    label       = test_label[correct_index[i]]
    im_vector   = test_image[correct_index[i]]
    im_matrix   = im_vector.reshape((size_row, size_col))

    plt.subplot(2, 5, i+1)
    plt.title(label)
    plt.imshow(im_matrix, cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
plt.show()

print("\nwrong")
for i in range(10):
    label       = test_predict[wrong_index[i]]
    im_vector   = test_image[wrong_index[i]]
    im_matrix   = im_vector.reshape((size_row, size_col))

    plt.subplot(2, 5, i+1)
    plt.title(label)
    plt.imshow(im_matrix, cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
plt.show()
